In [3]:
!pip install -q "pymc>=5" "arviz>=0.15"

In [10]:
import numpy as np
import pymc as pm
import arviz as az


publicity = np.array([
    1.5, 2.0, 2.3, 3.0, 3.5, 4.0, 4.5, 5.0, 5.5, 6.0,
    6.5, 7.0, 7.5, 8.0, 8.5, 9.0, 9.5, 10.0, 10.5, 11.0
])

sales = np.array([
    5.2, 6.8, 7.5, 8.0, 9.0, 10.2, 11.5, 12.0, 13.5, 14.0,
    15.0, 15.5, 16.2, 17.0, 18.0, 18.5, 19.5, 20.0, 21.0, 22.0
])


new_publicity = np.array([2.0, 6.0, 10.0])


with pm.Model() as model:

    x = pm.MutableData("publicity", publicity)


    alpha = pm.Normal("alpha", mu=0, sigma=20)
    beta  = pm.Normal("beta",  mu=0, sigma=10)
    sigma = pm.HalfNormal("sigma", sigma=5)


    mu = alpha + beta * x


    pm.Normal("sales", mu=mu, sigma=sigma, observed=sales)


    idata = pm.sample(2000, tune=1000, target_accept=0.9, random_seed=1)


# a)
print("\nRezumat posterior pentru coeficienți (alpha, beta, sigma):")
print(az.summary(idata, var_names=["alpha", "beta", "sigma"], hdi_prob=0.95))


# b)
coef_hdi = az.hdi(idata, var_names=["alpha", "beta"], hdi_prob=0.95)
print("\nHDI 95% pentru coeficienți (alpha, beta):")
print(coef_hdi)


# c)
posterior = idata.posterior

alpha_s = posterior["alpha"].stack(sample=("chain", "draw")).values
beta_s  = posterior["beta"].stack(sample=("chain", "draw")).values
sigma_s = posterior["sigma"].stack(sample=("chain", "draw")).values

rng = np.random.default_rng(1)

print("\nIntervale predictive 90% (5%, 95%) pentru new_publicity:")
for p in new_publicity:
    mu_p = alpha_s + beta_s * p
    y_pred = rng.normal(mu_p, sigma_s)
    lo, hi = np.percentile(y_pred, [5, 95])

    print(f"publicity = {p:4.1f} -> sales ~ {lo:5.2f} .. {hi:5.2f} (mii $)")


Output()


Rezumat posterior pentru coeficienți (alpha, beta, sigma):
        mean     sd  hdi_2.5%  hdi_97.5%  mcse_mean  mcse_sd  ess_bulk  \
alpha  3.376  0.222     2.930      3.806      0.006    0.005    1445.0   
beta   1.706  0.032     1.648      1.778      0.001    0.001    1431.0   
sigma  0.404  0.075     0.274      0.556      0.002    0.002    1561.0   

       ess_tail  r_hat  
alpha    1411.0    1.0  
beta     1349.0    1.0  
sigma    1412.0    1.0  

HDI 95% pentru coeficienți (alpha, beta):
<xarray.Dataset> Size: 80B
Dimensions:  (hdi: 2)
Coordinates:
  * hdi      (hdi) <U6 48B 'lower' 'higher'
Data variables:
    alpha    (hdi) float64 16B 2.93 3.806
    beta     (hdi) float64 16B 1.648 1.778
Attributes:
    created_at:                 2025-12-03T12:55:39.228214+00:00
    arviz_version:              0.22.0
    inference_library:          pymc
    inference_library_version:  5.26.1
    sampling_time:              6.75495982170105
    tuning_steps:               1000

Intervale pred